In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import panel as pn

from transformers import AutoTokenizer, BertModel

from icat.model import Model
from icat.anchors import DictionaryAnchor
pn.extension('vega')

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset="train")
test = fetch_20newsgroups(subset="test")

train_df = pd.DataFrame({"text": train["data"]})
test_df = pd.DataFrame({"text": test["data"]})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text_model = BertModel.from_pretrained("bert-base-uncased").to('cuda')

In [ ]:
text_model.encoder

In [ ]:
batch_size = 16

#def ossify_similarity_model(data: pd.DataFrame, anchorlist, target_text, text_col_name):
def bert_similarity_model(data: pd.DataFrame, anchor):
    anchorlist = anchor.container
    target_text = anchor.reference_texts[0] # TODO
    text_col_name = anchor.text_col
    
    # compute all of the embeddings for the full dataset as needed
    if not hasattr(anchorlist, "similarity_embeddings"):
        embedded_batches = []
        
        # run the tokenizer and model embedding on batches
        for batch in range(data.shape[0] // batch_size + 1):
            # compute range for this batch
            batch_start = batch * batch_size
            if batch == data.shape[0] // batch_size:
                batch_end = data.shape[0]
            else:
                batch_end = batch_start + batch_size
        
            batch_text = data[text_col_name].tolist()[batch_start:batch_end]
            
            tokenized = tokenizer(batch_text, return_tensors='pt', truncation=True, padding="max_length")["input_ids"].to('cuda').detach()
            sim_embeddings = text_model(tokenized).last_hidden_state.detach().cpu().numpy()
            embedded_batches.append(sim_embeddings)
            del tokenized
            
        # for batch in embedded_batches:
        #     print(batch.shape)
        # tokenized = tokenizer(data[text_col_name].tolist(), return_tensors='pt', padding=True)["input_ids"].to('cuda')
        # anchorlist.similarity_embeddings = model.model(tokenized).last_hidden_state.detach().cpu().numpy()
        anchorlist.similarity_embeddings = np.concatenate(embedded_batches, axis=0)
        anchorlist.similarity_embeddings = anchorlist.similarity_embeddings.mean(axis=1)
        print(anchorlist.similarity_embeddings.shape)
        
    tokenized_target = tokenizer(target_text, return_tensors='pt')["input_ids"].to('cuda')
    embedded_target = text_model.encoder(tokenized_target).last_hidden_state.detach().cpu().numpy()
    embedded_target = embedded_target.mean(axis=1)
    print(embedded_target.shape)
    
    similarities = cosine_similarity(embedded_target, anchorlist.similarity_embeddings)
    
    # some quick code to make the results spread out a bit more
    similarities *= 2
    similarities -= 1
    similarities[similarities < 0.5] = 0.0
    
    
    return pd.Series(similarities[0], index=data.index)

In [ ]:
model = Model(train_df, "text", similarity_functions=[bert_similarity_model])

In [ ]:
model.view

In [ ]:
model.anchor_list.anchors[0].container.model.similarity_functions[model.anchor_list.anchors[0].similarity_function](model.data.active_data, model.anchor_list.anchors[0])

In [ ]:
tokenized = tokenizer(model.data.active_data[model.text_col][0], return_tensors='pt', truncation=True, padding='max_length')["input_ids"].to('cuda')


In [ ]:
text_model(tokenized).last_hidden_state.detach().cpu().numpy()

In [ ]:
model.data.active_data[["_New Anchor"]].value_counts()

In [ ]:
model.anchor_list.anchors[0].widget.children[0].children[2].children[0].v_model

In [ ]:
model.anchor_list.anchors[1].new_id

In [ ]:
model.anchor_list.anchors[0].new_id

In [ ]:
model.anchor_list.anchors[0].widget.children[0].children[2].

In [ ]:
model.anchor_list.anchors[0].sim_function_options.items

In [ ]:
model.anchor_list.anchors[0]._populate_items()